In [16]:
import requests
import pandas as pd
import json
import time
import pickle

In [7]:
YOUR_API_KEY = ''

In [8]:
def get_data(offset):
    url = f"https://api.congress.gov/v3/member?api_key={YOUR_API_KEY}&format=json&offset={offset}&limit=250"
    response = requests.get(url)
    time.sleep(7)
    print(offset)
    return response.json()

In [9]:
def parse_member_data(member):
    bioguide_id = member["bioguideId"]
    name = member["name"]
    party = member["party"]
    state = member["state"]
    served = member["served"]
    house_served = served.get("House", [])
    senate_served = served.get("Senate", [])

    house_data = [{"bioguideId": bioguide_id, "name": name, "party": party, "state": state, "chamber": "House", "start": term["start"], "end": term.get("end", None)} for term in house_served]
    senate_data = [{"bioguideId": bioguide_id, "name": name, "party": party, "state": state, "chamber": "Senate", "start": term["start"], "end": term.get("end", None)} for term in senate_served]

    return house_data + senate_data

In [12]:
def main():
    offset = 0
    all_members_data = []

    while True:
        data = get_data(offset)
        members = data["members"]

        if not members:
            break

        for member_data in members:
            all_members_data.extend(parse_member_data(member_data["member"]))

        offset += 250

    df = pd.DataFrame(all_members_data)
    return(df)

In [13]:
df = main()

0
250
500
750
1000
1250
1500
1750
2000
2250
2500
2750


In [17]:
with open("congress_members.pkl", "wb") as file:
    pickle.dump(df, file)

In [18]:
with open("congress_members.pkl", "rb") as file:
    loaded_df = pickle.load(file)

print(loaded_df)

     bioguideId                 name       party         state chamber  start  \
0       S000394         Shuster, Bud  Republican  Pennsylvania   House   1973   
1       W000790  Warnock, Raphael G.  Democratic       Georgia  Senate   2021   
2       B001319    Britt, Katie Boyd  Republican       Alabama  Senate   2023   
3       H001086   Harshbarger, Diana  Republican     Tennessee   House   2021   
4       L000593          Levin, Mike  Democratic    California   House   2019   
...         ...                  ...         ...           ...     ...    ...   
2776    F000215     Florio, James J.  Democratic    New Jersey   House   1975   
2777    F000209     Flood, Daniel J.  Democratic  Pennsylvania   House   1945   
2778    F000209     Flood, Daniel J.  Democratic  Pennsylvania   House   1949   
2779    F000209     Flood, Daniel J.  Democratic  Pennsylvania   House   1955   
2780    F000208    Flippo, Ronnie G.  Democratic       Alabama   House   1977   

         end  
0     2001.0